In [ ]:
!pip install langchain langchain-community langchain-core langchain-experimental langchain-huggingface
!pip install faiss-cpu pdfplumber sentence-transformers openai gradio


INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-huggingface to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currentl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 101.8 MB/s eta 0:00:00


In [9]:
import os
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# OpenRouter API Key
os.environ["OPENROUTER_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# ========== Functions ==========
qa = None

def process_pdf(pdf_file):
    global qa
    status_msgs = []

    try:
        loader = PDFPlumberLoader(pdf_file.name)
        docs = loader.load()
        status_msgs.append(f"Loaded {len(docs)} pages from PDF.")

        # 2. Chunking
        status_msgs.append("Splitting text into chunks...")
        embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        text_splitter = SemanticChunker(embedder)
        documents = text_splitter.split_documents(docs)
        status_msgs.append(f"Created {len(documents)} chunks.")

        # 3. Vector DB
        status_msgs.append("Building FAISS vector database...")
        vector = FAISS.from_documents(documents, embedder)
        retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})
        status_msgs.append(" Vector database ready.")

        # 4. LLM
        status_msgs.append("Initializing LLM...")
        llm = ChatOpenAI(
              model="mistralai/mistral-7b-instruct",
              temperature=0.7,
              openai_api_base="https://openrouter.ai/api/v1",
              openai_api_key=os.environ["OPENROUTER_API_KEY"],
              request_timeout=180,
        )

        # 5. Prompt
        prompt_template = """
        You are an expert Python tutor.
Use ONLY the information provided in the context extracted from the PDF
"Python Programming – Wikibooks".

Rules:
- Do NOT use any outside knowledge.
- If the answer is not found in the context, say:
  "I could not find this information in the provided document."
- Keep answers short, clear, and accurate.
- Use examples ONLY if they appear in the context.
- Do not create your own examples.

Context:
{context}

Question:
{question}

Answer:

        """
        QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt_template)

        llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=False)
        document_prompt = PromptTemplate(
            input_variables=["page_content", "source"],
            template="Context:\ncontent:{page_content}\nsource:{source}",
        )

        combine_documents_chain = StuffDocumentsChain(
            llm_chain=llm_chain,
            document_variable_name="context",
            document_prompt=document_prompt,
        )

        qa = RetrievalQA(
            combine_documents_chain=combine_documents_chain,
            retriever=retriever,
            return_source_documents=True,
            verbose=False,
        )

        status_msgs.append("Ready for questions!")

    except Exception as e:
        return f"Error: {str(e)}"

    return "\n".join(status_msgs)

def ask_question(question):
    global qa
    if qa is None:
        return "Please upload a PDF first."

    try:
        result = qa({"query": question})

        # Handle different output formats safely
        if "result" in result:
            return result["result"]
        elif "answer" in result:
            return result["answer"]
        else:
            return f"⚠ Unexpected response: {result}"
    except Exception as e:
        return f"Error while answering: {str(e)}"


# ========== Gradio UI ==========
with gr.Blocks() as demo:
    gr.Markdown("## Q&A Bot")
    with gr.Row():
        pdf_input = gr.File(label="Upload PDF", file_types=[".pdf"])
        status = gr.Textbox(label="Status")
    upload_btn = gr.Button("Process PDF")
    with gr.Row():
        q_in = gr.Textbox(label="Ask a Question")
        q_out = gr.Textbox(label="Answer")
    ask_btn = gr.Button("Submit")

    upload_btn.click(process_pdf, inputs=[pdf_input], outputs=[status])
    ask_btn.click(ask_question, inputs=[q_in], outputs=[q_out])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://12b64f9bbb6f3dcf11.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
